In [ ]:
import urllib
import pandas as pd
import shapely.wkt
from shapely.geometry import shape
from shapely.errors import WKTReadingError
from shapely.geometry import Point
import json

In [ ]:
def get_all_organisations():
    params = urllib.parse.urlencode({
        "sql": f"""
        select organisation, name, entity as organisation_entity, statistical_geography
        from organisation
        """,
        "_size": "max"
        })
    url = f"https://datasette.planning.data.gov.uk/digital-land.csv?{params}"
    df = pd.read_csv(url)
    return df

def get_la_district_json(lpa_ref):
    params = urllib.parse.urlencode({
        "sql": f"""
        select json
        from entity
        where reference = '{lpa_ref}'
        """,
        "_size": "max"
        })
    url = f"https://datasette.planning.data.gov.uk/entity.csv?{params}"
    df = pd.read_csv(url)
    try:
        return df.loc[0,"json"]
    except KeyError:
        return None

def get_brownfield_sites_for_organisation(organisation_entity_number):
    params = urllib.parse.urlencode({
        "sql": f"""
        select json, point, reference, organisation_entity
        from entity
        where organisation_entity = '{organisation_entity_number}'
        """,
        "_size": "max"
        })
    url = f"https://datasette.planning.data.gov.uk/brownfield-land.csv?{params}"
    df = pd.read_csv(url)
    return df
 


In [ ]:
def get_LPA_multipolygon(reference):
    params = urllib.parse.urlencode({
        "sql": f"""
        select geometry
        from entity
        where reference = '{reference}'
        """,
        "_size": "max"
        })
    url = f"https://datasette.planning.data.gov.uk/entity.csv?{params}"
    df = pd.read_csv(url)
    try:
        return df.loc[0,"geometry"]
    except KeyError:
        return None

def get_site_point(collection_name, entity_number):
    params = urllib.parse.urlencode({
        "sql": f"""
        select point
        from entity
        where entity = '{entity_number}'
        """,
        "_size": "max"
        })
    url = f"https://datasette.planning.data.gov.uk/{collection_name}.csv?{params}"
    df = pd.read_csv(url)
    return df.loc[0,"point"]

def parse_wkt(value):
    try:
        geometry = shapely.wkt.loads(value)
    except WKTReadingError:
        try:
            geometry = shapely.wkt.loads(shape(json.loads(value)).wkt)
            return geometry, "invalid type geojson"
        except Exception:
            return None, "invalid WKT"
    return geometry, None


def make_point(point):
    if point.geom_type == "Point":
        return Point(point)
    else:
        print("Not a point")

In [ ]:
from urllib.request import urlopen


collection="brownfield_land"
df_lpa = get_all_organisations()
df_brownfield_sites_outside_lpa = pd.DataFrame(columns=["reference", "organisation", "organisation_name", "point", "google_maps_link", "admin_district"])
for lpa in df_lpa.itertuples():
    df_brownfield_sites = get_brownfield_sites_for_organisation(lpa.organisation_entity)
    df_brownfield_sites = df_brownfield_sites.merge(df_lpa, left_on="organisation_entity", right_on="organisation_entity")
    # display(df_brownfield_sites.head())
    if ("local-authority-eng" in lpa.organisation):
        #store the last 3 characters of lpa.organisation
        # la_district_ref = lpa.organisation[-3:]
        # la_district_json_string = get_la_district_json(la_district_ref)
        # if la_district_json_string is not None:
        #     la_district_json = json.loads(get_la_district_json(la_district_ref))
        #     geometry_ref = la_district_json["local-authority-district"]
            # print(geometry_ref)
            # print(lpa.statistical_geography)
        multipol = get_LPA_multipolygon(lpa.statistical_geography)
        if multipol is not None:
            area, issue = parse_wkt(multipol)
            # print(area, issue)
            for site in df_brownfield_sites.itertuples():
                # print(site)
                if (pd.isnull(site.point) == False):
                    # print (site.reference)
                    # print(site.point)
                    pt = shapely.wkt.loads(site.point)
                    if (pt.within(area) == False):
                        # print("Not within")
                        
                        url = f"https://api.postcodes.io/postcodes?lon={pt.x}&lat={pt.y}"
                        response = urlopen(url)
                        data = json.loads(response.read())
                        try:
                            admin_district = data["result"][0]["admin_district"]
                        except Exception:
                            admin_district = "None found"

                        google_maps_link = f"https://maps.google.com/?q={pt.y},{pt.x}"
                        pt_outside_boundary_row = {"reference": site.reference, "organisation": lpa.organisation, "organisation_name": lpa.name, "point": site.point, "google_maps_link": google_maps_link, "admin_district": admin_district}
                        
                        df_brownfield_sites_outside_lpa = pd.concat([df_brownfield_sites_outside_lpa, pd.DataFrame([pt_outside_boundary_row])] , ignore_index=True)
                        # print(pt)
                        # print(area)
                else:
                    pt_no_coord_row = {"reference": site.reference, "organisation": lpa.organisation, "organisation_name": lpa.name, "point": "No coordinate data"}
                    df_brownfield_sites_outside_lpa = pd.concat([df_brownfield_sites_outside_lpa, pd.DataFrame([pt_no_coord_row])] , ignore_index=True)
            




        

In [31]:
df_brownfield_sites_outside_lpa_sorted = df_brownfield_sites_outside_lpa["organisation"].value_counts().reset_index()
df_brownfield_sites_outside_lpa_sorted.to_csv("brownfield_sites_outside_lpa.csv")
display(df_brownfield_sites_outside_lpa_sorted)

,organisation,count
0,local-authority-eng:CHW,237
1,local-authority-eng:POR,232
2,local-authority-eng:NGM,207
3,local-authority-eng:COV,110
4,local-authority-eng:STN,103
...,...,...
172,local-authority-eng:NED,1
173,local-authority-eng:NDE,1
174,local-authority-eng:IPS,1
175,local-authority-eng:IOW,1
